In [ ]:

    
#     return filename
def plot_eeg_1(db_for_vis_C_, generated_samples_BA_, db_for_vis_D_, generated_samples_AB_, subject_idx, epoch, save_dir='plots'):
    num_trials = db_for_vis_C_.shape[0]
    fig, axs = plt.subplots(4, 2, figsize=(30, 22))
    
    # Sử dụng f-string để thay thế subject_idx trong đường dẫn
    save_dir = f"C:/Users/VU NGOC TAM/Downloads/gan1/1 times/26092024-kernel 2,4-sub {subject_idx}"

    plt.style.use('seaborn-white')
    for i in range(4):
        axs[i][0].plot(db_for_vis_C_[i, :], label='truth')
        axs[i][0].plot(generated_samples_BA_[i, :], label='generated')
        axs[i][0].set_title('EEG and NIRS->EEG (channel {} )'.format(i + 1), size=23)

        axs[i][1].plot(db_for_vis_D_[i, :], label='truth')
        axs[i][1].plot(generated_samples_AB_[i, :], label='generated')
        axs[i][1].set_title('NIRS and EEG->NIRS (channel {} )'.format(i + 1), size=23)

    axs[3][0].set_xlabel('Time samples', fontsize=23.0)
    axs[3][1].set_xlabel('Time samples', fontsize=23.0)

    legend_labels = ['ground truth', 'generated']
    for ax_row in axs:
        for ax in ax_row:
            ax.tick_params(axis='x', labelsize=23)
            ax.tick_params(axis='y', labelsize=23)
    
    plt.tight_layout()
    plt.legend(legend_labels, loc='center', fontsize=16, borderaxespad=0.01, bbox_to_anchor=(0.5125, 0.49), shadow=True, bbox_transform=plt.gcf().transFigure)
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Sử dụng f-string để lưu file với giá trị subject_idx và epoch
    filename = f"{save_dir}/plot_epoch_{epoch}.png"
    plt.savefig(filename, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    
    return filename


################################################################################################################################################ bắt đầu vô train


import os

# Function to save lists to txt files with directory specified
def save_to_txt(data, directory, filename):
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    file_path = os.path.join(directory, filename)
    with open(file_path, 'w') as f:
        for item in data:
            f.write("%s\n" % item)

# Compute test loss function
def compute_test_loss(model, xA, xB, iterr):
    loss_G_test = model.train_gen_test(xA, xB, iterr)
    loss_D_test = model.train_discrim(xA, xB, iterr)
    return loss_G_test, loss_D_test

# Main training loop
global_epoch = 400
total_epoch = 251
batch_size = 30
ckpt_dir = os.getcwd() + '/checkpoint/'
vis_num = 4

# Chia dữ liệu thành các nhóm 30 trials
num_trials_per_subject = 30
num_folds = eeg_normalize.shape[0] // num_trials_per_subject

# Tạo thư mục để lưu kết quả nếu chưa tồn tại
results_dir = "C:/Users/VU NGOC TAM/Downloads/gan1/result cross"
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

num_subjects = 29
num_trials_per_subject = 30

# Vòng lặp để chia dữ liệu thành các phần như mô tả
for subject_idx in range(num_subjects):
    tf.compat.v1.reset_default_graph()
    start_idx = subject_idx * num_trials_per_subject
    end_idx = start_idx + num_trials_per_subject
    
    # Chia tập kiểm thử và tập huấn luyện
    test_indices = np.arange(start_idx, end_idx)
    train_indices = np.setdiff1d(np.arange(eeg_normalize.shape[0]), test_indices)
    
    print(f"Đang test với subject thứ {subject_idx + 1}, trial {start_idx} đến {end_idx - 1}")
    print(f"Đang train với subject thứ {subject_idx + 1}, trial indices: {train_indices}")
    train_eeg = eeg_normalize[train_indices]
    train_nirs = nirs_normalized[train_indices]

    test_eeg = eeg_normalize[test_indices]
    test_nirs = nirs_normalized[test_indices]

    database_A = db.DBreader(train_eeg, batch_size=batch_size)
    database_B = db.DBreader(train_nirs, batch_size=batch_size)
    db_for_vis_A = train_eeg
    db_for_vis_B = train_nirs

    database_C = db.DBreader(test_eeg, batch_size=batch_size)
    database_D = db.DBreader(test_nirs, batch_size=batch_size)
    db_for_vis_C = test_eeg
    db_for_vis_D = test_nirs



    global_epoch = tf.Variable(0, trainable=False, name='global_step')
    global_epoch_increase = tf.compat.v1.assign(global_epoch, tf.add(global_epoch, 1))
    config = tf.compat.v1.ConfigProto(device_count={'GPU': 1, 'CPU': 12})
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    model = Discogan(sess, batch_size, size_=270)

    saver = tf.compat.v1.train.Saver(tf.compat.v1.global_variables())


    
    total_batch_A = database_A.total_batch
    total_batch_B = database_B.total_batch

    if total_batch_A > total_batch_B:
        total_batch = total_batch_B
    else:
        total_batch = total_batch_A

    discriminator_loss = []
    generator_loss = []
    corr_A_ABA = []
    corr_B_BAB = []
    total_corr_A_ABA = []
    total_corr_B_BAB = []
    corr_A_ABA_train_1_2 = []
    corr_B_BAB_train_1_2 = []
    total_corr_A_ABA_train_1_2 = []
    total_corr_B_BAB_train_1_2 = []
    test_losses_G_per_epoch = []
    test_losses_D_per_epoch = []
    
    corr_A_ABA_1= []
    total_corr_A_ABA_1= []
    corr_B_BAB_1= []
    total_corr_B_BAB_1= []
    corr_A_ABA_train_1= []
    total_corr_A_ABA_train_1= []
    corr_B_BAB_train_1= []
    total_corr_B_BAB_train_1= []


    sess.run(tf.compat.v1.global_variables_initializer())
    epoch = sess.run(global_epoch)
    print('HERE>>>>>>>>>>>>>>>>>>>')
    print(epoch)
    output_directory = f"C:/Users/VU NGOC TAM/Downloads/gan1/1 times/26092024-kernel 2,4-sub {subject_idx + 1}/output_txt"

    while True:
        if epoch == total_epoch:
            break
        for step in range(total_batch):
            input_A = database_A.next_batch()
            input_B = database_B.next_batch()

            input_C = database_C.next_batch()
            input_D = database_D.next_batch()

            if step % 2 == 0:
                loss_D = model.train_discrim_test(input_C, input_D, epoch * total_batch + step)  # Train Discriminator and get the loss value
                loss_G_test, loss_D_test = compute_test_loss(model, input_C, input_D, epoch * total_batch + step)
            loss_G = model.train_gen_test(input_C, input_D, epoch * total_batch + step)  # Train Generator and get the loss value

            if step % 100 == 0:
                print('Epoch: [', epoch, '/', total_epoch, '], ', 'Step: [', step, '/', total_batch, '], D_loss: ', loss_D, ', G_loss: ', loss_G)
                discriminator_loss.append(loss_D)
                generator_loss.append(loss_G)
                test_losses_G_per_epoch.append(loss_G_test)
                test_losses_D_per_epoch.append(loss_D_test)

            if step % 500 == 0:
                generated_samples_AB = model.sample_generate(db_for_vis_C, 'AB',batch_size=30)
                generated_samples_ABA = model.sample_generate(db_for_vis_C, 'ABA',batch_size=30)
                generated_samples_BA = model.sample_generate(db_for_vis_D, 'BA',batch_size=30)
                generated_samples_BAB = model.sample_generate(db_for_vis_D, 'BAB',batch_size=30)

                generated_samples_AB_ = np.mean(np.array(generated_samples_AB), axis=0)
                generated_samples_ABA_ = np.mean(np.array(generated_samples_ABA), axis=0)
                generated_samples_BA_ = np.mean(np.array(generated_samples_BA), axis=0)
                generated_samples_BAB_ = np.mean(np.array(generated_samples_BAB), axis=0)

                db_for_vis_C_ = np.mean(np.array(db_for_vis_C), axis=0)
                db_for_vis_D_ = np.mean(np.array(db_for_vis_D), axis=0)

                img_for_vis_AB = np.concatenate([db_for_vis_C, generated_samples_AB, generated_samples_ABA], axis=2)
                img_for_vis_BA = np.concatenate([db_for_vis_D, generated_samples_BA, generated_samples_BAB], axis=2)

                corr_a_test = correlation(db_for_vis_C_, generated_samples_ABA_)
                corr_A_ABA.append(corr_a_test)
                total_corr_A_ABA.append(np.mean(corr_a_test))
                corr_b_test = correlation(db_for_vis_D_, generated_samples_BAB_)
                corr_B_BAB.append(corr_b_test)
                total_corr_B_BAB.append(np.mean(corr_b_test))

                corr_a_train_1_2 = correlation(db_for_vis_C_, generated_samples_BA_)
                corr_A_ABA_train_1_2.append(corr_a_train_1_2)
                total_corr_A_ABA_train_1_2.append(np.mean(corr_a_train_1_2))
                corr_b_train_1_2 = correlation(db_for_vis_D_, generated_samples_AB_)
                corr_B_BAB_train_1_2.append(corr_b_train_1_2)
                total_corr_B_BAB_train_1_2.append(np.mean(corr_b_train_1_2))


        epoch = sess.run(global_epoch_increase)
        save_to_txt(test_losses_G_per_epoch, output_directory, "test_losses_G_per_epoch.txt")
        save_to_txt(test_losses_D_per_epoch, output_directory, "test_losses_D_per_epoch.txt")
        save_to_txt(total_corr_A_ABA_train_1_2, output_directory, "total_corr_A_ABA_train_1_Correlation.txt")
        save_to_txt(total_corr_B_BAB_train_1_2, output_directory, "total_corr_B_BAB_train_1_Correlation.txt")


        trial_index = 9
        # plot_eeg_2(db_for_vis_C_, generated_samples_ABA_, db_for_vis_D_, generated_samples_BAB_)
        if epoch % 50 == 0:
            plot_eeg_1(db_for_vis_C_, generated_samples_BA_, db_for_vis_D_, generated_samples_AB_, subject_idx + 1, epoch)
########################################################################################################################################  vẽ hình thôi nha a
        # plot_3_fig(db_for_vis_C_, generated_samples_AB_, generated_samples_ABA_, db_for_vis_D_, generated_samples_BA_, generated_samples_BAB_, save_dir='plots')
        base_dir = f"C:/Users/VU NGOC TAM/Downloads/gan1/1 times/26092024-kernel 2,4-sub {subject_idx + 1}"
        subject_dir = os.path.join(base_dir, f"{subject_idx + 1}")

        # Tạo thư mục nếu chưa tồn tại
        if not os.path.exists(subject_dir):
            os.makedirs(subject_dir)
        plotting_data = [test_losses_D_per_epoch,test_losses_G_per_epoch,total_corr_A_ABA_train_1_2, total_corr_B_BAB_train_1_2]

        graph_titles = ['Discriminator loss test 1 times',
                             'Generator loss test 1 times',
                        'Mean of Correlation between EEG and NIRS->EEG',
                        'Mean of Correlation between NIRS and EEG->NIRS']
        y_axis_titles = list(['Loss'] * 2 +
                             ['Correlation Coefficient'] * 4)
        fig, axs = plt.subplots(int(len(plotting_data) / 2), 2, figsize=(24, 12), sharey='row', facecolor='w', edgecolor='k')
        fig.subplots_adjust(hspace=.30, wspace=.20)
        axs = axs.ravel()
        for i, (title, data) in enumerate(zip(graph_titles, plotting_data)):
            if np.shape(np.shape(data))[0] == 1:
                axs[i].plot(data)
            else:
                for result in data:
                    axs[i].plot(result)
            axs[i].set_title(title, fontweight="bold", size=22)
            axs[i].set_xlabel('Number of Epochs', fontsize=20.0)
            axs[i].set_ylabel(y_axis_titles[i], fontsize=20.0)
            axs[i].tick_params(axis='y', which='both', left=True, right=False, labelleft=True, labelsize=18)
            axs[i].tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True, labelsize=18)
        # fig.savefig(f'loss_corrcoef_subject_{subject_idx + 1}_{total_epoch}epochs.png', pad_inches=0.015, bbox_inches='tight')
        loss_corrcoef_path = os.path.join(subject_dir, f"loss_corrcoef_subject_{subject_idx + 1}_{total_epoch}epochs.png")
        fig.savefig(loss_corrcoef_path, pad_inches=0.015, bbox_inches='tight')

        plotting_data_test = [test_losses_D_per_epoch, test_losses_G_per_epoch, total_corr_A_ABA, total_corr_B_BAB]
        graph_titles_test = ['Discriminator loss test 2 times',
                             'Generator loss test 2 times', 'Mean of Correlation between EEG and EEG->NIRS->EEG',
                             'Mean of Correlation between NIRS and NIRS->EEG->NIRS']
        y_axis_titles = list(['Loss'] * 2 +
                             ['Correlation Coefficient'] * 4)
        fig, axs = plt.subplots(int(len(plotting_data_test) / 2), 2, figsize=(24, 12), sharey='row', facecolor='w', edgecolor='k')
        fig.subplots_adjust(hspace=.30, wspace=.20)
        axs = axs.ravel()
        for i, (title, data) in enumerate(zip(graph_titles_test, plotting_data_test)):
            if np.shape(np.shape(data))[0] == 1:
                axs[i].plot(data)
            else:
                for result in data:
                    axs[i].plot(result)
            axs[i].set_title(title, fontweight="bold", size=22)
            axs[i].set_xlabel('Number of Epochs', fontsize=20.0)
            axs[i].set_ylabel(y_axis_titles[i], fontsize=20.0)
            axs[i].tick_params(axis='y', which='both', left=True, right=False, labelleft=True, labelsize=18)
            axs[i].tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True, labelsize=18)
        # fig.savefig(f'test_loss_subject_{subject_idx + 1}_{total_epoch}epochs.png', pad_inches=0.015, bbox_inches='tight')
        
        
        test_loss_path = os.path.join(subject_dir, f"test_loss_subject_{subject_idx + 1}_{total_epoch}epochs.png")
        fig.savefig(test_loss_path, pad_inches=0.015, bbox_inches='tight')
        plt.close(fig)
################################################################################################################################################
    sess.close()
    tf.compat.v1.reset_default_graph()



